## Testing WDRC

This notebook tests WDRC on a running OSP process.

You will need to start osp running in a terminal.

In [ ]:
# make Jupyter use the whole width of the browser
from IPython.display import Image, display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Pull in all the necessary python modules

from IPython.display import Audio
from osp_control import OspControl, plot_audio
import os
import json
import numpy as np
import time

In [ ]:
# Connect to OSP process
osp = OspControl()  # connects to a local process

# set the number of bands and mute mic
osp.send({"method": "set", "data": {"num_bands": 11}})
osp.send_chan({"alpha": 1})

In [ ]:
# set left G50, but keep right G50=0 so we can measure the difference

g50_val = 15
osp.send({"method": "set", "data": {"left": {"g50": g50_val}, "right": {"g50": 0}}})

# osp.send({"method": "set", "data": {"left": {"g80": -5}, "right": {"g50": 0}}})
# TODO: replicate the capability for g65/cr

In [ ]:
# verify values.  not necessary
res = osp.send({"method": "get"})
print("Left  g50=", res['left']['g50'])
print("Right g50=", res['right']['g50'])

In [ ]:
# set record file name
filename = os.path.join(os.getcwd(), 'WDRCtest')
osp.send_chan({'audio_rfile': filename})

In [ ]:
#start recording output
osp.send_chan({"audio_record": 2})

# play mono file
# afile = os.path.join(os.getcwd(), '12345.wav')
afile = "Freping/MPEG_es03_s.wav"
osp.send_chan({"audio_filename": afile, "audio_play": 1, "audio_alpha": 1})

# record for 4 seconds
time.sleep(4)

#stop recording and playback
osp.send_chan({"audio_record": 0, "audio_play": 0})

In [ ]:
# read the saved recording data
with open(filename, "rb") as f:
    data = f.read()
data = np.frombuffer(data, dtype=np.float32)
data = np.reshape(data, (2, -1), 'F')

In [ ]:
# plot it
plot_audio('', data[0], f'g50={g50_val}', data[1], 'g50=0')

You should see the left channel has a compressed dynamic range.

In [ ]:
Audio(data, rate=48000, normalize=True)